## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
import os
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
file_to_load = os.path.join("Resources", "WeatherPy_database.csv")
city_data_df = pd.read_csv(file_to_load)
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Weather Description
0,0,Kieta,PG,-6.2167,155.6333,78.75,84,46,6.35,light rain
1,1,Rikitea,PF,-23.1203,-134.9692,73.02,77,2,14.00,clear sky
2,2,Cherskiy,RU,68.7500,161.3000,30.04,74,59,4.36,broken clouds
3,3,Camacha,PT,33.0833,-16.3333,72.82,69,20,12.66,few clouds
4,4,Tautira,PF,-17.7333,-149.1500,77.02,75,25,18.30,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
max_temp = float(input("What is the maximum temperature you would want on your vacation? "))
min_temp = float(input("What is the minimum temperature you would want on your vacation? "))

What is the maximum temperature you would want on your vacation? 85
What is the minimum temperature you would want on your vacation? 70


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_temps_df = city_data_df.loc[(city_data_df['Max Temp'] >= min_temp) & (city_data_df['Max Temp'] <= max_temp)]
preferred_temps_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Weather Description
0,0,Kieta,PG,-6.2167,155.6333,78.75,84,46,6.35,light rain
1,1,Rikitea,PF,-23.1203,-134.9692,73.02,77,2,14.00,clear sky
3,3,Camacha,PT,33.0833,-16.3333,72.82,69,20,12.66,few clouds
4,4,Tautira,PF,-17.7333,-149.1500,77.02,75,25,18.30,scattered clouds
5,5,Butaritari,KI,3.0707,172.7902,82.90,70,72,15.01,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_temps_df.isnull().sum()

City_ID                        0
City                           0
Country                        3
Lat                            0
Lng                            0
Max Temp                       0
Humidity                       0
Cloudiness                     0
Wind Speed                     0
Current Weather Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_temps_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Weather Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head()


,City,Country,Max Temp,Current Weather Description,Lat,Lng,Hotel Name
0,Kieta,PG,78.75,light rain,-6.2167,155.6333,
1,Rikitea,PF,73.02,clear sky,-23.1203,-134.9692,
3,Camacha,PT,72.82,few clouds,33.0833,-16.3333,
4,Tautira,PF,77.02,scattered clouds,-17.7333,-149.1500,
5,Butaritari,KI,82.90,broken clouds,3.0707,172.7902,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
           
hotel_df.head()

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Weather Description,Lat,Lng,Hotel Name
0,Kieta,PG,78.75,light rain,-6.2167,155.6333,Uruna Bay Retreat
1,Rikitea,PF,73.02,clear sky,-23.1203,-134.9692,People ThankYou
3,Camacha,PT,72.82,few clouds,33.0833,-16.3333,Hotel Porto Santo & Spa
4,Tautira,PF,77.02,scattered clouds,-17.7333,-149.1500,Pension Oaoa
5,Butaritari,KI,82.90,broken clouds,3.0707,172.7902,Isles Sunset Lodge


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Weather Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [15]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 40.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))